In [ ]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

Saving Aquaponics Water Quality Data.csv to Aquaponics Water Quality Data.csv
Saving GE_RefractoryAlloyScreeningDataset_FINAL.csv to GE_RefractoryAlloyScreeningDataset_FINAL.csv
User uploaded file "Aquaponics Water Quality Data.csv" with length 116738 bytes
User uploaded file "GE_RefractoryAlloyScreeningDataset_FINAL.csv" with length 4706 bytes


In [ ]:
import pandas as pd
import numpy as np
import os
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
import joblib

def clean_col_names(df):
    """
    Cleans column names of a pandas DataFrame by replacing spaces with
    underscores and removing special characters.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with cleaned column names.
    """
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'[^a-zA-Z0-9_]', '', col.strip().replace(' ', '_'))
        new_cols.append(new_col)
    df.columns = new_cols
    return df

def load_aquaponics_data(data_path):
    """
    Loads and performs initial preprocessing on the Aquaponics Water Quality Data.

    Args:
        data_path (str): The path to the Aquaponics CSV data file.

    Returns:
        pd.DataFrame: The loaded and initially preprocessed DataFrame.

    Raises:
        FileNotFoundError: If the specified data file does not exist.
        Exception: For errors during file reading or initial cleaning.
    """
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"Data file not found: {data_path}")

    print(f"Loading Aquaponics data from {data_path}...")
    try:
        df = pd.read_csv(data_path)
        df = clean_col_names(df)
        print(f"Successfully loaded {os.path.basename(data_path)} with shape {df.shape}")
        return df
    except Exception as e:
        raise Exception(f"Error loading {os.path.basename(data_path)}: {e}")


def preprocess_aquaponics_data(df):
    """
    Performs detailed preprocessing on the Aquaponics Water Quality Data DataFrame.

    Includes datetime conversion, feature extraction, handling missing values,
    dropping irrelevant columns, and identifying feature types.

    Args:
        df (pd.DataFrame): The raw Aquaponics DataFrame.

    Returns:
        pd.DataFrame: The preprocessed DataFrame.
    """
    print("\nPreprocessing Aquaponics data...")
    # Convert 'created_at' to datetime and extract time-based features
    if 'created_at' in df.columns:
        # Using errors='coerce' to handle potential parsing issues
        df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
        # Drop rows where created_at could not be parsed
        df.dropna(subset=['created_at'], inplace=True)
        if not df.empty:
            df['hour'] = df['created_at'].dt.hour
            df['day_of_week'] = df['created_at'].dt.dayofweek
            # Added more granular time features
            df['month'] = df['created_at'].dt.month
            df['day'] = df['created_at'].dt.day
            df['year'] = df['created_at'].dt.year
            df['weekofyear'] = df['created_at'].dt.isocalendar().week.astype(int)

            # Add lag features for relevant numeric columns
            # Identify numeric columns excluding time features just added and potential targets
            numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
            lag_features_to_consider = [col for col in numeric_cols if col not in ['hour', 'day_of_week', 'month', 'day', 'year', 'weekofyear']]

            # Sort by time before creating lag features
            df.sort_values(by='created_at', inplace=True)

            for col in lag_features_to_consider:
                 # Add 1-period lag feature
                 df[f'{col}_lag1'] = df[col].shift(1)
                 # Add 2-period lag feature (optional, can extend)
                 df[f'{col}_lag2'] = df[col].shift(2)


        df.drop('created_at', axis=1, inplace=True)


    # Drop entry_id if it exists and is not useful as a feature
    if 'entry_id' in df.columns:
        df.drop('entry_id', axis=1, inplace=True)

    # Drop 'Unnamed' columns with mostly NaNs
    df = df.loc[:, ~df.columns.str.startswith('Unnamed_')]

    # Identify potential categorical columns that are currently objects
    # Excluding 'Notes' as it's likely free text and not easily encoded
    categorical_cols = df.select_dtypes(include='object').columns.tolist()
    categorical_cols = [col for col in categorical_cols if col not in ['Notes', 'Day', 'Date', 'Time']] # Exclude original date/time columns if created_at was used

    print(f"Identified potential categorical columns for encoding: {categorical_cols}")

    # Drop columns that are not features or targets and are not being encoded
    all_possible_features_targets = categorical_cols + df.select_dtypes(include=np.number).columns.tolist() + ['Notes'] # Include Notes to explicitly drop later if needed
    cols_to_drop = [col for col in df.columns if col not in all_possible_features_targets]
    if cols_to_drop:
        print(f"Dropping non-feature/target columns: {cols_to_drop}")
        df.drop(columns=cols_to_drop, inplace=True)


    print(f"Aquaponics DataFrame shape after preprocessing: {df.shape}")
    print("Aquaponics DataFrame info after preprocessing:")
    df.info()
    return df, categorical_cols # Return categorical columns list

def train_and_evaluate_model(df, target, categorical_features, models_dir, model_name_prefix):
    """
    Trains and evaluates a Random Forest Regressor model for a specific target variable.

    Includes preprocessing pipeline with imputation, scaling, one-hot encoding,
    and optional target normalization. Uses GridSearchCV for hyperparameter tuning.

    Args:
        df (pd.DataFrame): The input DataFrame containing features and the target.
        target (str): The name of the target variable column.
        categorical_features (list): A list of categorical feature column names.
        models_dir (str): Directory to save the trained model.
        model_name_prefix (str): Prefix to use for saving the model file (e.g., 'aquaponics').

    Returns:
        dict: A dictionary containing evaluation metrics (MSE, R2) and best parameters,
              or None if training/evaluation is skipped or fails.
    """
    print(f"\nTraining model for: {target}")

    # Check if the target column exists and has non-null values
    if target not in df.columns or df[target].dropna().empty:
        print(f"Skipping {target}: Target column not found or contains no non-null values.")
        return None

    # Drop rows where the target variable is NaN
    df_target = df.dropna(subset=[target]).copy()

    # Dynamically identify features after dropping NaNs in the target
    # Exclude the target column and the 'Notes' column
    features = [col for col in df_target.columns if col != target and col != 'Notes']

    # Separate numeric and categorical features from the *actual* features available in df_target
    numeric_features = df_target[features].select_dtypes(include=np.number).columns.tolist()
    categorical_features = df_target[features].select_dtypes(include='object').columns.tolist()


    if not features: # Check if any features remain after exclusions
        print(f"Skipping {target}: No valid features found after dropping NaNs and excluding target/Notes.")
        return None

    X = df_target[features]
    y = df_target[target]

    if X.empty or y.empty:
        print(f"Skipping {target}: X or y is empty after preprocessing or filtering.")
        return None

    # Implement Target Normalization (e.g., PowerTransformer)
    # Only apply if the target variable has positive values and some variance
    target_transformer = None
    original_y = y.copy() # Keep original for inverse transform

    if y.min() >= 0 and y.var() > 1e-6: # Check for non-negative values and variance
        print(f"Applying PowerTransformer to target: {target}")
        target_transformer = PowerTransformer(method='yeo-johnson') # Yeo-Johnson handles zero/negative
        try:
             # Need to reshape y for the transformer
            y = target_transformer.fit_transform(y.values.reshape(-1, 1)).flatten()
        except ValueError as e:
             print(f"Could not apply PowerTransformer to {target}: {e}. Skipping transformation.")
             y = original_y # Revert to original if transformation fails
             target_transformer = None # Don't use transformer if it failed


    # Create preprocessing pipelines for numeric and categorical features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')), # Impute before scaling
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')), # Impute before encoding
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])


    # Create a column transformer to apply different transformations to different columns
    # Include both numeric and categorical transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)],
        remainder='passthrough') # Keep other columns (if any, though 'drop' was used earlier)


    # Define the full pipeline including preprocessing and the regressor
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))])


    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit the model with GridSearchCV for hyperparameter tuning
    try:
        param_grid = {
            'regressor__n_estimators': [100, 200, 300], # Test more values
            'regressor__max_depth': [None, 10, 20, 30], # Test more depths
            'regressor__min_samples_split': [2, 5, 10] # Test min samples split
        }
        # Using k-fold cross-validation within GridSearchCV
        grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1) # Increased folds to 5
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_ # Get the best model from GridSearchCV
        print(f"Best parameters for {target}: {grid_search.best_params_}")
        print(f"Best cross-validation R2 score for {target}: {grid_search.best_score_:.4f}")


    except ValueError as e:
        print(f"Error fitting pipeline or GridSearchCV for {target}: {e}")
        return None
    except Exception as e:
         print(f"An unexpected error occurred during model training for {target}: {e}")
         return None


    # Save the model
    model_filename = os.path.join(models_dir, f"{model_name_prefix}_{target}.joblib")
    try:
        joblib.dump(best_model, model_filename)
        print(f"Model for {target} saved to {model_filename}")
    except IOError as e:
        print(f"Error saving model for {target} to {model_filename}: {e}")
        # Continue evaluation even if saving fails

    # Evaluate the best model on the test set
    try:
        y_pred = best_model.predict(X_test)
        # Inverse transform predictions if target was transformed
        if target_transformer:
            y_pred_original_scale = target_transformer.inverse_transform(y_pred.reshape(-1, 1)).flatten()
            # Also inverse transform the test set target for correct evaluation
            y_test_original_scale = target_transformer.inverse_transform(y_test.reshape(-1, 1)).flatten()
            mse = mean_squared_error(y_test_original_scale, y_pred_original_scale)
            r2 = r2_score(y_test_original_scale, y_pred_original_scale)
            print(f"Evaluation on original scale for {target} (after inverse transform):")
        else:
            # If no transformation, evaluate on the scale of y_test
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)


        print(f"Test Set Mean Squared Error for {target}: {mse:.4f}")
        print(f"Test Set R2 Score for {target}: {r2:.4f}")
        return {"mse": mse, "r2": r2, "best_params": grid_search.best_params_, "cv_r2": grid_search.best_score_}
    except Exception as e:
        print(f"Error evaluating model for {target}: {e}")
        return None


if __name__ == "__main__":
    # --- Configuration ---
    aquaponics_data_filename = "Aquaponics Water Quality Data.csv"
    data_directory = "/content" # Assuming uploaded files are in /content
    aquaponics_data_path = os.path.join(data_directory, aquaponics_data_filename)
    models_dir = "/content/trained_models_aquaponics" # Separate models directory for aquaponics
    aquaponics_preprocessed_path = "/content/preprocessed_aquaponics_data.csv"

    # Create models directory if it doesn't exist
    os.makedirs(models_dir, exist_ok=True)

    # --- Process Aquaponics Data ---
    aquaponics_df = None
    try:
        aquaponics_df_raw = load_aquaponics_data(aquaponics_data_path)
        aquaponics_df, aquaponics_categorical_features = preprocess_aquaponics_data(aquaponics_df_raw)

        # Save the preprocessed Aquaponics data
        try:
            aquaponics_df.to_csv(aquaponics_preprocessed_path, index=False)
            print(f"Preprocessed Aquaponics data saved to {aquaponics_preprocessed_path}")
        except IOError as e:
            print(f"Error saving preprocessed Aquaponics data: {e}")

        # Define target variables for Aquaponics data as requested
        aquaponics_target_variables = ["Ammonia_ppm", "pH", "Dissolved_Oxygen_ppm", "Water_Temp_F", "Nitrate_ppm"]

        # Identify ALL potential features before filtering for each target
        # Exclude target variables, Notes, and Unnamed columns from the raw features pool
        all_potential_features = [col for col in aquaponics_df.columns if col not in aquaponics_target_variables and col != 'Notes' and not col.startswith('Unnamed_')]

        # We don't strictly need aquaponics_features list here anymore as features are determined dynamically in the training function


        print("\nStarting ML model development for Aquaponics data...")
        aquaponics_evaluation_results = {}
        for target_var in aquaponics_target_variables:
             # Pass categorical features list to train_and_evaluate_model
             results = train_and_evaluate_model(aquaponics_df, target_var, aquaponics_categorical_features, models_dir, "aquaponics")
             if results:
                 aquaponics_evaluation_results[target_var] = results

        print("\nML model development for Aquaponics data complete.")
        print("\nAquaponics Evaluation Results:")
        for target, metrics in aquaponics_evaluation_results.items():
            print(f"{target}: Test MSE = {metrics['mse']:.4f}, Test R2 = {metrics['r2']:.4f}, CV R2 = {metrics['cv_r2']:.4f}, Best Params = {metrics['best_params']}")

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during Aquaponics data processing: {e}")


    print("\nOverall process complete.")

Loading Aquaponics data from /content/Aquaponics Water Quality Data.csv...
Successfully loaded Aquaponics Water Quality Data.csv with shape (366, 29)

Preprocessing Aquaponics data...
Identified potential categorical columns for encoding: ['Recorded_by_Remy_RomoValdez_x__not_recorder_0__recorder', 'Javier_Mollinedo_x__not_recorder_0__recorder', 'Recorded_by_Joe_Tocci_x__not_recorder_0__recorder', 'Recorded_by_Ellison_Montgomery_x__not_recorder_0__recorder']
Dropping non-feature/target columns: ['Day', 'Date', 'Time']
Aquaponics DataFrame shape after preprocessing: (366, 12)
Aquaponics DataFrame info after preprocessing:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 12 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Recorded_by_Remy_RomoValdez_x__not_recorder_0__recorder     366 non-null    object 


/tmp/ipython-input-18-117864035.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=cols_to_drop, inplace=True)


Best parameters for Ammonia_ppm: {'regressor__max_depth': None, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 200}
Best cross-validation R2 score for Ammonia_ppm: 0.8100
Model for Ammonia_ppm saved to /content/trained_models_aquaponics/aquaponics_Ammonia_ppm.joblib
Evaluation on original scale for Ammonia_ppm (after inverse transform):
Test Set Mean Squared Error for Ammonia_ppm: 0.1474
Test Set R2 Score for Ammonia_ppm: 0.7731

Training model for: pH
Applying PowerTransformer to target: pH
Best parameters for pH: {'regressor__max_depth': 10, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 200}
Best cross-validation R2 score for pH: 0.5796
Model for pH saved to /content/trained_models_aquaponics/aquaponics_pH.joblib
Evaluation on original scale for pH (after inverse transform):
Test Set Mean Squared Error for pH: 0.0703
Test Set R2 Score for pH: 0.7415

Training model for: Dissolved_Oxygen_ppm
Applying PowerTransformer to target: Dissolved_Oxygen_ppm
Best 